In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pickle
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model


In [3]:
df = pd.read_csv('Data/dataset_futbol_limpio_1.0.csv', index_col = 'Player')
df

,Nation,Pos,Squad,Comp,Age,MP,Starts,Min,90s,Goals,...,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
Player,,,,,,,,,,,,,,,,,,,,,
Brenden Aaronson,USA,MF,Leeds United,Premier League,22,20,19,1596,17.7,1,...,0.17,2.54,0.51,0.0,0.0,0.00,4.86,0.34,1.19,22.2
Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,35,22,22,1980,22.0,0,...,0.05,0.18,1.59,0.0,0.0,0.00,6.64,2.18,1.23,64.0
Himad Abdelli,FRA,MF,Angers,Ligue 1,23,14,8,770,8.6,0,...,0.00,1.05,1.40,0.0,0.0,0.00,8.14,0.93,1.05,47.1
Salis Abdul Samed,GHA,MF,Lens,Ligue 1,22,20,20,1799,20.0,1,...,0.00,0.35,0.80,0.0,0.0,0.05,6.60,0.50,0.50,50.0
Laurent Abergel,FRA,MF,Lorient,Ligue 1,30,15,15,1165,12.9,0,...,0.00,0.23,2.02,0.0,0.0,0.00,6.51,0.31,0.39,44.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Szymon ?urkowski,POL,MF,Fiorentina,Serie A,25,2,0,32,0.4,0,...,0.00,2.50,0.00,0.0,0.0,0.00,5.00,2.50,2.50,50.0
Szymon ?urkowski,POL,MF,Spezia,Serie A,25,1,0,8,0.1,0,...,0.00,0.00,0.00,0.0,0.0,0.00,30.00,0.00,0.00,0.0
Martin Ødegaard,NOR,MF,Arsenal,Premier League,24,20,20,1684,18.7,8,...,0.05,2.51,0.70,0.0,0.0,0.00,5.19,0.48,0.70,40.9


In [4]:
df_numerico = df.select_dtypes(include=[np.number])

# Calcular la matriz de correlación
correlacion = df_numerico.corr()

# Mostrar solo las correlaciones de la columna 'Goals' con las demás columnas
correlacion_goals = correlacion['Goals'].drop('Goals') 

print("Correlaciones con 'Goals':\n", correlacion_goals)

Correlaciones con 'Goals':
 Age        0.054098
MP         0.406454
Starts     0.388006
Min        0.378071
90s        0.377935
             ...   
OG        -0.036824
Recov     -0.121223
AerWon     0.017194
AerLost    0.046832
AerWon%   -0.030022
Name: Goals, Length: 116, dtype: float64


-La mayoría de los datos de las estadísticas estan calculados en promedio por partido(90 minutos), esto a la hora de entrenar nuestro modelo puede crear un conflicto y provocar que nuestro modelo no aprenda bien, es por ello que vamos a ajustar todas las columnas del dataframe, para posteriormente entrenar nuestro modelo. 

In [5]:
# Función para ajustar las estadísticas por 90 minutos
def ajustar_estadisticas(df, columnas, minutos_columna='Min'):
    df_copy = df.copy()  # Evitar advertencias al modificar el DataFrame
    for col in columnas:
        df_copy[col] = df_copy[col] * df_copy[minutos_columna] / 90  # Ajustar cada estadística por el total de minutos
    return df_copy

-Vamos a definir 2 funciones, la primera tomará unos parámetros y entrenara el modelo con los datos de las columnas de esos parámetros, una vez entrenado el modelo, se almacenara en el equipo, para posteriormente ser utilizado.
La segunda función cogerá ese modelo ya entrenado y lo utilizará para predecir un parámetro que le habremos metido también como input, el cual será el parámetro a predecir.

-Esto se puede utilizar para predecir cuántos goles y asistencias, va a producir un determinado jugador de cara a sus promedios por 90 minutos.


In [6]:

#Esta funcion se encraga de entrenar nuestro modelo y guardarlo.
def model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador):
    df_filtrado = df[df['Pos'] == posicion_jugador]

    # Ajustar las estadísticas de los jugadores por 90 minutos
    df_filtrado = ajustar_estadisticas(df_filtrado, caracteristicas_entrenamiento, minutos_columna='Min')
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_filtrado[caracteristicas_entrenamiento])
    y = df_filtrado[caracteristica_predecir]

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

    y_pred = modelo.predict(X_test)
    error = mean_squared_error(y_test, y_pred)
    print(f'Error cuadrático medio: {error}')
    ruta_completa_modelo = f'/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelo_RegLin.pkl'
    with open(ruta_completa_modelo, 'wb') as archivo_modelo:
        pickle.dump((modelo, scaler), archivo_modelo)

    print(f'Modelo entrenado y guardado correctamente ')
    return ruta_completa_modelo

# Función para cargar el modelo y predecir para un jugador ficticio
def load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados, ruta_completa_modelo):
    with open(ruta_completa_modelo, 'rb') as archivo_modelo:
        modelo, scaler = pickle.load(archivo_modelo)

    jugador_nuevo_caracteristicas = [valor * minutos_jugados / 90 for valor in jugador_nuevo_caracteristicas]
    valores_caracteristicas_df = np.array(jugador_nuevo_caracteristicas).reshape(1, -1)
    valores_caracteristicas_scaled = scaler.transform(valores_caracteristicas_df)

    prediccion = modelo.predict(valores_caracteristicas_scaled)
    os.remove('/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelo_RegLin.pkl')
    print(f'Modelo eliminado')

    return prediccion



-Vamos a proceder a testear nuestro modelo, para ello vamos a meterle unso datos reales de jugadores del propio dataset, y veremo a ver si predice mas o menos bien los goles reales que ha metido.

In [7]:
# Ejemplo de uso
Nombre_jugador = 'Miguel Almirón'
caracteristicas_entrenamiento = ['Shots', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'ShoDist', 'ShoFK', 'ShoPK','PKatt', 'PasTotAtt', 'PasTotCmp%', 'PasTotDist', 'PasTotPrgDist', 'PasShoCmp', 'PasShoAtt', 'PasShoCmp']
caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [2.23, 0.74, 31.9, 0.21, 0.67, 16.8,0, 0,0, 29.3, 35.5, 82.5, 401.3, 99.6, 18.2, 20.1 ]  # Por 90 minutos
minutos_jugados_ficticio = 1816  # Minutos totales jugados por el jugador ficticio
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador ficticio utilizando nuestro modelo: {prediccion_goles[0]}")

Error cuadrático medio: 0.14783659943811792
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador ficticio utilizando nuestro modelo: 8.711994938463992


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [8]:
# Ejemplo de uso con jugador real.
Nombre_jugador = 'Miguel Almirón'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    2.33, 0.74, 31.9, 0.21, 0.67, 16.8, 0, 0, 0, 29.3, 35.5, 82.5, 401.3, 99.6, 
    18.2, 20.1, 90.4, 8.61, 10.5, 82.1, 1.14, 1.63, 69.7, 0.05, 0.84, 1.78, 1.14, 
    0.1, 4.8, 35.5, 34.2, 1.14, 0, 0, 0.15, 0.94, 0.69, 0.45, 0, 0, 0, 29.3, 
    0.25, 1.09, 3.07, 2.18, 0.1, 0.35, 0.3, 0.15, 0, 0.45, 0.15, 0.05, 0.2, 
    0.05, 0, 0, 1.78, 1.19, 0.59, 0.74, 0.45, 0.74, 1.24, 60, 0.5, 1.14, 0.1, 
    1.04, 0.35, 2.13, 0.45, 0, 47.3, 0.79, 5.05, 15.2, 27.9, 4.46, 47.3, 2.92, 
    1.09, 37.3, 1.39, 47.5, 29.9, 199.6, 85.6, 2.82, 1.44, 0.94, 1.58, 0.89, 
    33.1, 9.5, 0.05, 0, 0, 0.69, 1.29, 0.4, 0.94, 1.19, 0, 0, 0, 5.1, 0.2, 0.45, 30.8
]

minutos_jugados_ficticio = 1816  # Minutos totales jugados por el jugador ficticio
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador ficticio con Regresión Lineal: {prediccion_goles[0]}")

Error cuadrático medio: 0.2160854809991706
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador ficticio con Regresión Lineal: 9.876800542519687


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
# Ejemplo de uso con jugador real.
Nombre_jugador = 'Folarín Balogun'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    3.33, 1.58, 47.5, 0.17, 0.36, 15.2, 0.06, 0.23, 0.23, 10.5, 16.1, 65.3, 146.9, 34.1, 
    6.21, 7.97, 78, 3.22, 4.97, 64.8, 0.4, 0.96, 41.2, 0.06, 0.85, 0.85, 0.62, 0, 1.98, 
    16.1, 14.7, 1.24, 0, 0.17, 0, 0.68, 0, 0, 0, 0, 0, 10.5, 0.17, 0.62, 2.15, 1.3, 0, 
    0.45, 0.23, 0.17, 0, 0.34, 0.11, 0, 0.11, 0, 0.11, 0, 0.28, 0.11, 0, 0.17, 0.11, 
    0.06, 0.51, 11.1, 0.45, 0.9, 0.06, 0.85, 0.06, 0.34, 0.62, 0, 28.6, 0.62, 2.71, 
    11.2, 15, 5.88, 28.4, 2.94, 1.02, 34.6, 1.58, 53.8, 21.2, 134.4, 52.4, 1.92, 0.62, 
    1.36, 2.82, 2.6, 21, 8.31, 0.17, 0, 0, 0.9, 1.92, 0.96, 0.68, 0.11, 0.11, 0, 0, 
    2.94, 0.51, 2.03, 20
]


minutos_jugados_ficticio = 1591  # Minutos totales jugados por el jugador ficticio
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador ficticio con Regresión Lineal: {prediccion_goles[0]}")

Error cuadrático medio: 0.2160854809991706
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador ficticio con Regresión Lineal: 14.179066465107738


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [10]:
# Ejemplo de uso con jugador real.
Nombre_jugador = 'Erling Haaland'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    3.85, 1.98, 51.4, 0.3, 0.58, 12.6, 0, 0.22, 0.22, 11.7, 15.9, 73.7, 147.5, 28.8, 
    6.76, 8.46, 79.9, 3.57, 4.67, 76.5, 0.27, 0.33, 83.3, 0.16, 0.93, 0.49, 0.77, 0, 
    1.87, 15.9, 13.6, 2.09, 0, 0.11, 0, 0.27, 0.11, 0, 0, 0, 0, 11.7, 0.22, 0.49, 
    2.03, 1.43, 0, 0.11, 0.22, 0.22, 0.05, 0.38, 0.33, 0, 0, 0, 0.05, 0, 0.11, 0.05, 
    0, 0.05, 0.05, 0, 0.05, 0, 0.05, 0.38, 0.11, 0.27, 0.11, 0.22, 0.55, 0, 24.2, 
    0.6, 1.43, 9.56, 13.2, 7.53, 24, 0.77, 0.27, 35.7, 0.44, 57.1, 12.6, 55.7, 20.5, 
    1.04, 0.27, 0.93, 1.43, 0.99, 18.1, 5.05, 0.16, 0, 0, 0.82, 0.82, 0.27, 0.27, 
    0.05, 0.05, 0, 0, 1.1, 1.37, 1.26, 52.1
]



minutos_jugados_ficticio = 1636  # Minutos totales jugados por el jugador ficticio
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador ficticio con Regresión Lineal: {prediccion_goles[0]}")

Error cuadrático medio: 0.2160854809991706
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador ficticio con Regresión Lineal: 22.748804138321212


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


-Ejemplos de uso con jugadores ficticios:


In [8]:
# Ejemplo de uso con jugador inventado.
Nombre_jugador = 'Carlinhos Brown'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    1.22, 1.28, 34.5, 0.13, 0.36, 14.2, 0.06, 0.23, 0.23, 10.5, 16.1, 65.3, 146.9, 34.1, 
    6.21, 7.97, 78, 3.22, 4.97, 64.8, 0.4, 0.96, 41.2, 0.06, 0.85, 0.85, 0.62, 0, 1.98, 
    16.1, 14.7, 1.24, 0, 0.17, 0, 0.68, 0, 0, 0, 0, 0, 10.5, 0.17, 0.62, 2.15, 1.15, 0, 
    0.45, 0.23, 0.17, 0, 0.34, 0.10, 0, 0.11, 0, 0.11, 0, 0.28, 0.11, 0, 0.17, 0.10, 
    0.06, 0.51, 11.1, 0.45, 0.9, 0.06, 0.85, 0.06, 0.34, 0.62, 0, 28.6, 0.62, 2.71, 
    10.2, 15, 5.88, 28.4, 2.94, 1.02, 31.6, 1.58, 53.8, 21.2, 134.4, 52.4, 1.92, 0.62, 
    1.36, 2.82, 2.6, 21, 8.31, 0.17, 0, 0, 0.9, 1.92, 0.96, 0.68, 0.11, 0.11, 0, 0, 
    2.44, 0.51, 2.03, 20
]


minutos_jugados_ficticio = 2200  # Supongamos que el jugador ficticio va a jugar 2200 minutos en la temporada.
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador {Nombre_jugador} con Regresión Lineal: {prediccion_goles[0]}")


Error cuadrático medio: 0.2160854809991706
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador Carlinhos Brown con Regresión Lineal: 15.434898306919452


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [26]:
# Ejemplo de uso con jugador inventado.
Nombre_jugador = 'Francois Gallardo'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    0.57, 0.65, 34.5, 0.13, 0.36, 14.2, 0.06, 0.23, 0.23, 10.5, 16.1, 65.3, 146.9, 34.1, 
    2.21, 3.97, 78, 3.22, 1.97, 55.8, 0.4, 0.96, 41.2, 0.06, 0.85, 0.85, 0.62, 0, 1.98, 
    16.1, 14.7, 1.24, 0, 0.17, 0, 0.68, 0, 0, 0, 0, 0, 10.5, 0.17, 0.62, 2.15, 1.15, 0, 
    0.45, 0.23, 0.17, 0, 0.34, 0.10, 0, 0.10, 0, 0.11, 0, 0.28, 0.11, 0, 0.17, 0.10, 
    0.06, 0.51, 11.1, 0.7, 0.9, 0.06, 0.85, 0.06, 0.34, 0.62, 0, 28.6, 0.62, 2.71, 
    10.2, 15, 5.88, 29.4, 2.94, 1.02, 31.6, 1.58, 33.8, 21.2, 134.4, 52.4, 1.92, 0.22, 
    1.36, 2.82, 2.6, 21, 8.31, 0.17, 0, 0, 0.3, 1.92, 0.96, 0.68, 0.11, 0.11, 0, 0, 
    2.44, 0.51, 2.03, 20
]


minutos_jugados_ficticio = 2200  # Supongamos que el jugador ficticio va a jugar 2200 minutos en la temporada.
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)
print(f"Predicción de goles para el jugador {Nombre_jugador} con Regresión Lineal: {prediccion_goles[0]}")


Error cuadrático medio: 0.2160854809991706
Modelo entrenado y guardado correctamente 
Modelo eliminado
Predicción de goles para el jugador Francois Gallardo con Regresión Lineal: 3.095542838169453


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [14]:

# Función para entrenar el modelo con una red neuronal potente usando Keras
def model_player_scouting_train_nn(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador):
    df_filtrado = df[df['Pos'] == posicion_jugador]

    # Ajustar las estadísticas de los jugadores por 90 minutos
    df_filtrado = ajustar_estadisticas(df_filtrado, caracteristicas_entrenamiento, minutos_columna='Min')
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_filtrado[caracteristicas_entrenamiento])
    y = df_filtrado[caracteristica_predecir].values

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Crear el modelo 
    modelo = Sequential()

    # Añadir capas (haremos un modelo sencillo)
    modelo.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dense(32, activation='relu'))
    modelo.add(Dense(16, activation='relu'))
    # Capa de salida (regresión -> una neurona de salida)
    modelo.add(Dense(1, activation='linear'))

    # Compilar el modelo con el optimizador Adam y la función de pérdida MSE
    modelo.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Early stopping para detener el entrenamiento cuando no hay mejora()
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Entrenar el modelo (Hemos puesto pocos epochs , por que normlamnete el early stopping lo paraba antes de los 100)
    modelo.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = modelo.predict(X_test)
    error = mean_squared_error(y_test, y_pred)
    print(f'Error cuadrático medio (Red Neuronal): {error}')

    # Guardar 
    ruta_completa_modelo = f'/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelos/Modelo_nn.h5'
    modelo.save(ruta_completa_modelo)


    with open('/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelos/scaler_nn', 'wb') as archivo_scaler:
        pickle.dump(scaler, archivo_scaler)

    print(f'Modelo de Red Neuronal entrenado y guardado correctamente.')
    return ruta_completa_modelo

# Función para cargar el modelo y predecir para un jugador ficticio
def load_model_and_test_nn(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados, ruta_completa_modelo):

    modelo = load_model(ruta_completa_modelo)

    with open('/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelos/scaler_nn', 'rb') as archivo_scaler:
        scaler = pickle.load(archivo_scaler)

    # Ajustar las características del jugador ficticio a sus minutos jugados
    jugador_nuevo_caracteristicas = [valor * minutos_jugados / 90 for valor in jugador_nuevo_caracteristicas]
    valores_caracteristicas_df = np.array(jugador_nuevo_caracteristicas).reshape(1, -1)

    # Escalar las características
    valores_caracteristicas_scaled = scaler.transform(valores_caracteristicas_df)

    # Predicción con el modelo cargado.
    prediccion = modelo.predict(valores_caracteristicas_scaled)
    os.remove('/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelos/scaler_nn')
    os.remove('/Users/hugo/Documents/Universidad/Proyecto Ciencia de Datos/Proyecto1/Modelos/Modelo_nn.h5')
    print(f'Modelo eliminado')

    return prediccion


In [15]:

# Ejemplo de uso con jugador real.
Nombre_jugador = 'Erling Haaland'
caracteristicas_entrenamiento = [
    "Shots", "SoT", "SoT%", "G/Sh", "G/SoT", "ShoDist", "ShoFK", "ShoPK", "PKatt", 
    "PasTotCmp", "PasTotAtt", "PasTotCmp%", "PasTotDist", "PasTotPrgDist", 
    "PasShoCmp", "PasShoAtt", "PasShoCmp%", "PasMedCmp", "PasMedAtt", "PasMedCmp%", 
    "PasLonCmp", "PasLonAtt", "PasLonCmp%", "Assists", "PasAss", "Pas3rd", "PPA", 
    "CrsPA", "PasProg", "PasAtt", "PasLive", "PasDead", "PasFK", "TB", "Sw", "PasCrs", 
    "TI", "CK", "CkIn", "CkOut", "CkStr", "PasCmp", "PasOff", "PasBlocks", "SCA", 
    "ScaPassLive", "ScaPassDead", "ScaDrib", "ScaSh", "ScaFld", "ScaDef", "GCA", 
    "GcaPassLive", "GcaPassDead", "GcaDrib", "GcaSh", "GcaFld", "GcaDef", "Tkl", 
    "TklWon", "TklDef3rd", "TklMid3rd", "TklAtt3rd", "TklDri", "TklDriAtt", "TklDri%", 
    "TklDriPast", "Blocks", "BlkSh", "BlkPass", "Int", "Tkl+Int", "Clr", "Err", 
    "Touches", "TouDefPen", "TouDef3rd", "TouMid3rd", "TouAtt3rd", "TouAttPen", 
    "TouLive", "ToAtt", "ToSuc", "ToSuc%", "ToTkl", "ToTkl%", "Carries", "CarTotDist", 
    "CarPrgDist", "CarProg", "Car3rd", "CPA", "CarMis", "CarDis", "Rec", "RecProg", 
    "CrdY", "CrdR", "2CrdY", "Fls", "Fld", "Off", "Crs", "TklW", "PKwon", "PKcon", 
    "OG", "Recov", "AerWon", "AerLost", "AerWon%"
]

caracteristica_predecir = 'Goals'
jugador_nuevo_caracteristicas = [
    3.85, 1.98, 51.4, 0.3, 0.58, 12.6, 0, 0.22, 0.22, 11.7, 15.9, 73.7, 147.5, 28.8, 
    6.76, 8.46, 79.9, 3.57, 4.67, 76.5, 0.27, 0.33, 83.3, 0.16, 0.93, 0.49, 0.77, 0, 
    1.87, 15.9, 13.6, 2.09, 0, 0.11, 0, 0.27, 0.11, 0, 0, 0, 0, 11.7, 0.22, 0.49, 
    2.03, 1.43, 0, 0.11, 0.22, 0.22, 0.05, 0.38, 0.33, 0, 0, 0, 0.05, 0, 0.11, 0.05, 
    0, 0.05, 0.05, 0, 0.05, 0, 0.05, 0.38, 0.11, 0.27, 0.11, 0.22, 0.55, 0, 24.2, 
    0.6, 1.43, 9.56, 13.2, 7.53, 24, 0.77, 0.27, 35.7, 0.44, 57.1, 12.6, 55.7, 20.5, 
    1.04, 0.27, 0.93, 1.43, 0.99, 18.1, 5.05, 0.16, 0, 0, 0.82, 0.82, 0.27, 0.27, 
    0.05, 0.05, 0, 0, 1.1, 1.37, 1.26, 52.1
]



minutos_jugados_ficticio = 1636  # Minutos totales jugados por el jugador ficticio
posicion_jugador = 'FW'  # Posición del jugador ficticio

# Ajustar y entrenar el modelo solo con jugadores de la misma posición y guardar la ruta del modelo
ruta_modelo = model_player_scouting_train_nn(df, caracteristicas_entrenamiento, caracteristica_predecir, posicion_jugador)

# Ejecutar la predicción para el jugador ficticio ajustando por sus minutos jugados
prediccion_goles = load_model_and_test_nn(caracteristicas_entrenamiento, jugador_nuevo_caracteristicas, minutos_jugados_ficticio, ruta_modelo)

print(f"Predicción de goles para el jugador ficticio con Red Neuronal: {prediccion_goles[0]}")

/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-20 20:23:10.764054: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-10-20 20:23:10.764079: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-20 20:23:10.764086: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-20 20:23:10.764132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-20 20:23:10.764147: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271]

Epoch 1/50


2024-10-20 20:23:12.196373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 296ms/step - loss: 13.9125 - val_loss: 7.0522
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 5.0415 - val_loss: 3.9755
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.7773 - val_loss: 2.0562
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.3614 - val_loss: 1.6881
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.1281 - val_loss: 2.2498
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.1600 - val_loss: 1.7678
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.2820 - val_loss: 1.2641
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.8358 - val_loss: 1.1589
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6762 - val_loss: 0.9297
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4788 - val_loss: 0.8743
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4273 - val_loss: 0.9658
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2913 - val

Error cuadrático medio (Red Neuronal): 0.5375259084615072
Modelo de Red Neuronal entrenado y guardado correctamente.


/Users/hugo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Modelo eliminado
Predicción de goles para el jugador ficticio con Red Neuronal: [25.148357]
